In [1]:
#pip install pybluez
#pip install pyserial
# python -m pip install -e git+https://github.com/pybluez/pybluez.git#egg=pybluez
# on linux : sudo apt-get install bluetooth libbluetooth-dev
# sudo python3 -m pip install pybluez
# I didn't have a specific microsoft c++ package.

import serial
import time
from bluetooth import *
# Scan for available Bluetooth devices
nearby_devices = discover_devices(lookup_names=True)
print("Scanning for devices...")
print("Found {} devices.".format(len(nearby_devices)))

# List devices
for index, device in enumerate(nearby_devices):
    addr, name = device
    print("{}: {} - {}".format(index, addr, name))

# Select your ESP32 device
selection = int(input("Select device index: "))
addr, name = nearby_devices[selection]
print("You have selected {}".format(name))

# Replace with your ESP32's Bluetooth address
# Example: '01:23:45:67:89:AB'
esp32_bt_address = 'b0:a7:32:2a:8a:46'

# Connect to the ESP32 over Bluetooth
port = 1  # The standard port for Bluetooth Serial on ESP32
sock = BluetoothSocket(RFCOMM)
sock.connect((esp32_bt_address, port))
print("Connected to {}".format(name))


# f = open("read_write_file.txt", "r")  #open the file for reading input


def sending_info(command): 
    # Now let's send data
    while True:
        # Write a string to the ESP32
        # line = f.readline().strip() # reading the file for input
        
        # data_to_send = line 
        # For Manual Input:
        # data_to_send = input("Enter data to send: ")
        data_to_send = str(command)
        if data_to_send.lower() == 'exit':
            break
        sock.send(data_to_send + '\n')  # Send the data with a newline character
        time.sleep(1)  # Wait a bit, necessary for some devices

    # sock.close()
    # print("Disconnected")

Scanning for devices...
Found 7 devices.
0: 41:42:0B:57:00:E8 - IQPODZ06
1: 14:98:77:E7:8F:5D - Yun Lei’s AirPods
2: E8:AB:FA:37:CD:07 - TAOTRONICS SoundSurge 60
3: 14:3F:A6:09:D9:EE - WH-1000XM4
4: B0:BE:83:49:CE:13 - MacBook Pro (2)
5: D4:D8:53:DD:8C:80 - JASHVIR1
6: 30:BB:7D:F3:A8:40 - makishima


ValueError: invalid literal for int() with base 10: ''

### sending the data to the esp32

In [ ]:
%run mediapipe_formatting_data/opencv_video_cutting.ipynb

In [ ]:
import cv2
import numpy as np
from numpy import asarray
import pandas as pd
import tensorflow as tf
import mediapipe as mp
import os
import csv
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
import time
import streamlit as st
from matplotlib import pyplot as plt
from PIL import Image

In [ ]:

def make_prediction(model_path, labels, csv_file):
    my_model = tf.keras.models.load_model(model_path, compile=False)
    my_model.compile(loss=SparseCategoricalCrossentropy(), optimizer=Adam(), metrics=["accuracy"])
    predictions = {n: 0 for n in labels}
    csv = pd.read_csv(csv_file)
    # print(csv)
    coords = np.array(csv)
    for frame in coords:
        new_frame = tf.expand_dims(frame,0)
        preds = my_model.predict(new_frame)
        pred_value = np.argmax(preds)
        # for i in range(len(preds[0])):
        #     print(preds[0][i])
        #     predictions[saved_classes[i]] += preds[0][i]
        #     total += preds[0][i]
        predictions[labels[pred_value]] += 1
        # print(preds)
    final_prediction = max(predictions, key=predictions.get)
    return predictions, final_prediction



# #writing the information to the parent file
# # https://blog.enterprisedna.co/python-write-to-file/
# fobj = open(os.path.join(os.pardir, "commands.csv"), "w")
# fobj.write(str(number[prediction[1]]))

# fobj.close()

In [ ]:
def run():
    # model_path = "Computer_Vision/training_computer_vision/model/model.107-0.98"# "D:/Personnel/Other learning/Programming/Personal_projects/ASL_Language_translation/training_models/mediapipe/Simple-Dense-Layers/regularized-4-labels.10-0.68"
    # model_path = r"/mnt/DATA/Personnel/Other learning/Programming/Personal_projects/3_Hackathons_with_buddies/UTRAHACKS2024/Computer_Vision/training_computer_vision/model/model.107-0.98"
    model_path = r"D:\Personnel\Other learning\Programming\Personal_projects\3_Hackathons_with_buddies\UTRAHACKS2024\Computer_Vision\training_computer_vision\model\model.107-0.98"
    labels = ["start", "turn", "grapple"]

    # csv_file = "/Users/aly/Documents/Programming/Apps/Machine Learning/ASL Converter/training_models/mediapipe/demo_test/demo.csv"
    csv_file="demo.csv"

    input("TRY ME OUT!! ")
    # extract_coordinates()
    open_cam("demo.csv")
    time.sleep(0.5)
    prediction = make_prediction(model_path, labels, csv_file)

    # !clear the csv file after each use
    # opening the file with w+ mode truncates the file
    f = open(csv_file, "w+")
    f.close()


    number = {"start": 0, "turn": 1, "grapple": 2}
    print(type(prediction))
    print(prediction)
    print(prediction[1])
    #sending the info to the esp:
    sending_info(number[prediction[1]])
    

In [ ]:
while input("Type x to stop") != 'x':
    run()
    time.sleep(5)


# sending_info("exit")
sock.close()
print("Disconnected")